In [1]:
import os
import re
import csv
import sys
import math
import heapq
import shutil
import queue as Q
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba
import nltk
from model import TrieNode

In [ ]:
folder = os.path.exists('blocks')
if not folder:
    os.makedirs('blocks')

In [2]:
ori_path = '动漫'
files= os.listdir(ori_path)

In [3]:
def construct1(intToJuji, intToFanju):
    i = 0
    j = -1000
    k = 0
    terms_id = dict()
    for root, dirs, files in os.walk(ori_path, topdown=True):
        if '\.ipynb_checkpoints' in root:
            continue
        if len(files) == 0:
            j+=1000
            i=0
            intToFanju[j] = root
        else:
            i+=1
            key = j+i
            intToJuji[key] = root
            for name in files:
                f = open(root+"\\"+ name, errors='ignore')
                lines = f.readlines()
                texts_s,texts_l = arrayTodict(lines)
                counts = cut3(texts_s)
            for word in counts:
                terms_id[word] = terms_id.get(word, 0) + counts[word]
    return terms_id

In [26]:
def construct(intToJuji, intToFanju):
    i = 0
    j = -1000
    k = 0
    terms_id = dict()
    doc_len = dict()
    for root, dirs, files in os.walk(ori_path, topdown=True):
        if '\.ipynb_checkpoints' in root:
            continue
        if len(files) == 0:
            if j>0:
                build_c(j, terms_id)
                terms_id = dict()
            j+=1000
            i=0
            intToFanju[j] = root
        else:
            i+=1
            key = j+i
            intToJuji[key] = root
            counts = dict()
            for name in files:
                f = open(root+"\\"+ name, errors='ignore')
                lines = f.readlines()
                texts_l = arrayTodict(lines)
                counts = dict_union(counts, cut1(texts_l))
            for word in counts:
                if word.isalpha() and counts[word]<3:
                    continue
                if key in doc_len:
                    doc_len[key] += counts[word]
                else:
                    doc_len[key] = counts[word]
                if word in terms_id:
                    terms_id[word].append((key,counts[word]))
                else: 
                    terms_id[word] = [(key,counts[word])]
    build_c(j, terms_id)
    terms_id = dict()
    return doc_len

In [4]:
intToJuji = dict()
intToFanju= dict()

In [29]:
len(intToJuji)

30306

In [27]:
doc_len = construct(intToJuji, intToFanju)

In [42]:
len(doc_len)

27050

In [35]:
list1 = sorted(doc_len.items(), key=lambda item: item[1])

In [37]:
list1.reverse()

In [39]:
    path = "doclen_new.txt"
    with open(path, 'w', newline='') as f:
        for key in doc_len:
            f.write(str(key))
            f.write('-')
            f.write(str(doc_len[key]))
            f.write('\n')
    f.close()

In [149]:
small_words = sorted(small_words.items(),key = lambda item: -item[1])

In [6]:
def arrayTodict(lines):
    word_dict_s={}
    word_dict_l={}
    for line in lines:
        line=shorten(line.strip('\n').replace(' ', ''))
        if len(line)<2:
            continue
        if len(line)<5:
            if word_dict_s.get(line):
                word_dict_s[line] += 1
            else:
                word_dict_s[line] = 1
            continue
        if word_dict_l.get(line):
            word_dict_l[line] += 1
        else:
            word_dict_l[line] = 1
    return word_dict_s, word_dict_l

In [24]:
def arrayTodict(lines):
    word_dict_l={}
    for line in lines:
        line=shorten(line.strip('\n').replace(' ', ''))
        if len(line)<2 or bool(re.search(r'\d', line)):
            continue
        if word_dict_l.get(line):
            word_dict_l[line] += 1
        else:
            word_dict_l[line] = 1
    return word_dict_l

In [11]:
def dict_union(d1, d2):
    keys = d1.keys() | d2.keys()
    temp = {}
    for key in keys:
        temp[key] = sum([d.get(key,0) for d in (d1, d2)])
    return temp

In [10]:
def cut1(word_dict):
    counts = dict()
    for line in word_dict:
        num = word_dict[line]
        words = jieba.lcut(line)
        if len(words) > 5:
            continue
        for word in words:
            if len(word) == 1:
                continue
            else:
                counts[word] = counts.get(word, 0) + num
    return counts

In [10]:
def cut3(word_dict):
    counts = dict()
    for line in word_dict:
        num = word_dict[line]
        counts[line] = counts.get(line, 0) + num
    return counts

In [11]:
def cut2(word_dict):
    counts = dict()
    for line in word_dict:
        num = word_dict[line]
        if len(line)<5:
            counts[line] = counts.get(line, 0) + num
            continue
        words = jieba.lcut(line)
        for word in words:
            if len(word) == 1:
                continue
            else:
                counts[word] = counts.get(word, 0) + num
    return counts

In [12]:
def build_c(i, terms_id):
    list1 = []
    path = "chunks_big/chunk_"+str(i)+".txt"
    list1 = sorted(terms_id.items(), key=lambda item: item[0])
    with open(path, 'w', newline='') as f:
        for key in list1:
            list2= []
            f.write(str(key[0]))
            for word in key[1]:
                f.write(',')
                f.write(str(word[0]))
                f.write('-')
                f.write(str(word[1]))
            f.write('\n')
    f.close()

In [9]:
def make_dirs():
    # Create a new folder to generate inverted indexes
    folder1 = os.path.exists('chunks')
    folder2 = os.path.exists('blocks')
    if not folder1:
        os.makedirs('chunks')
    if not folder2:
        os.makedirs('blocks')

In [87]:
make_dirs()

In [94]:
shorten('2333')

'233'

In [95]:
shorten('23333333')

'233'

In [97]:
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
def updateFPtree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        # 判断items的第一个结点是否已作为子结点
        inTree.children[items[0]].inc(count)
    else:
        # 创建新的分支
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新相应频繁项集的链表，往后添加
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    # 递归
    if len(items) > 1:
        updateFPtree(items[1::], inTree.children[items[0]], headerTable, count)

def createFPtree(word_dict, cutted, minSup=1):
    headerTable = {}
    for trans in word_dict:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    freqItemSet = set(headerTable.keys()) # 满足最小支持度的频繁项集
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] # element: [count, node]

    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        # dataSet：[element, count]
        localD = {}
        for item in tranSet:
            if item in freqItemSet: # 过滤，只取该样本中满足最小支持度的频繁项
                localD[item] = headerTable[item][0] # element : count
        if len(localD) > 0:
            # 根据全局频数从大到小对单样本排序
            orderedItem = [v[0] for v in localD.items()]
            # 用过滤且排序后的样本更新树
            updateFPtree(orderedItem, retTree, headerTable, count)
    return retTree, headerTable

In [171]:
print ("/".join(jieba.cut("吉良吉影觉得很赞")))

吉良吉影/觉得很赞


In [14]:
new_words = []
f = open("new_words_all.txt")
lines = f.readlines()
for line in lines:
    split = line.strip().split(',')
    new_words.append((split[0],int(split[1])))
f.close()

In [15]:
new_words[0]

('233', 251716)

In [16]:
for key_word in new_words:
    if key_word[1] > 2:
        jieba.add_word(key_word[0], freq = key_word[1]*10, tag = None)

In [46]:
N = len(intToJuji)

In [47]:
N

30306

In [36]:
f = open('juji.txt','w', encoding="utf-8")
for juji in intToJuji:
    f.write(str(juji))
    f.write(',')
    f.write(intToJuji[juji])
    f.write('\n')
f.close()

In [34]:
f.close()

In [54]:
def build_blocks_heap(N):
    # SPIMI
    reader = txt_reader('chunks_big')
    heap = []
    for i in range(len(reader)):
        next_line = reader[i].readline().strip().split(',')
        heapq.heappush(heap, (next_line[0], next_line[1:], i))

    terms_id = dict()
    j = 0
    next_term = ''
    try:
        while not len(heap) == 0:
            next_index = heapq.heappop(heap)
            judge = (next_index[0] == next_term)
            next_term = next_index[0]
            next_line = reader[next_index[2]].readline()
            if next_line:
                next_line= next_line.strip().split(',')
                heapq.heappush(heap, (next_line[0], next_line[1:], next_index[2]))
            else:
                reader[next_index[2]].close()
            if (sys.getsizeof(terms_id) > 5000000 and not judge):
                j = j + 1
                build_b(j, terms_id, N)
                terms_id = dict()
            if next_term in terms_id:
                terms_id[next_term].extend(next_index[1])
            else: 
                terms_id[next_term] = next_index[1]
    finally:
        j=j+1
        build_b(j, terms_id, N)            

In [56]:
def build_b(i, terms_id, N):
    path = "blocks_big/block_"+ str(i) +".csv"
    with open(path, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            for key in terms_id:
                if len(terms_id[key]) == 1:
                    continue
                list2= []
                list2.append(key)
                idf = math.log(N /(len(terms_id[key])+1), 10)
                list2.append(idf)
                list2.extend(terms_id[key])
                writer.writerow(list2)
        finally:
            f.close()

In [57]:
build_blocks_heap(N)

In [39]:
def txt_reader(path):
    fileList = os.listdir(path)
    files_reader = dict()
    i = 0
    for file in fileList:
        if(file.split('.')[-1] == "txt"):
            name = path+'/'+file
            files_reader[i] = open(name, "r")
            i = i + 1
    return files_reader

In [59]:
path = 'blocks_big'
fileList = os.listdir(path)
files_reader = dict()
for file in fileList:
    if(file.split('.')[-1] == "csv"):
        name = path+'/'+file
        com = re.compile(r"_(\d+).csv", re.M)
        find = com.findall(name)
        i = int(find[0])
        files_reader[i] = name